# Mixing Station Workspace

Update the .csv file to include the desired constituent electrolytes and their volume ratios, making sure they match up to the correct numbered location on the physical workspace. The columns in the .csv file correspond to the following.. 

| Name | Volume (uL) | Starting Volume (mL) | Aspirate Speed (mbar/s) | Aspirate Constant (mbar/ml) |
| --- | --- | --- | --- | --- |

Run the code block below and confirm that your total volume is correct.

In [1]:
from src.robot_controller import experiment_setup
experiment = experiment_setup.experiment("electrolye_recipe.csv", "COM12", "COM11", SIM=True)

INFO:root:No serial connection to gantry established.
INFO:root:No serial connection to pipette established.
INFO:root:Reading CSV file..


,Name,Volume (uL),Starting Volume (mL),Aspirate Speed (mbar/s),Aspirate Constant (mbar/ml)
0,O2,50.0,10.0,1.0,1.0
1,N2,20.0,10.0,1.0,1.0
2,CH4,200.0,10.0,1.0,1.0
3,NH4,100.0,10.0,1.0,1.0
4,CO2,0.0,10.0,1.0,1.0
5,CH3CH3,70.0,10.0,1.0,1.0
6,OH,10.0,10.0,1.0,1.0
7,NaOH,0.0,10.0,1.0,1.0
8,PO4,25.0,10.0,1.0,1.0
9,Au,35.0,10.0,1.0,1.0


INFO:root:Experiment will result in a total electrolyte volume of 0.51ml
INFO:root:Experiment ready to begin: 22/11/2024 15:05:06


## Run Experiment

In [2]:
experiment.run(N=4)

INFO:root:Creating electrolyte mixture #0..
INFO:root:Moving to O2..
INFO:root:Pipette charged.
INFO:root:Dropping Pipette into O2..
INFO:root:Aspiration complete.
INFO:root:50.0ul extracted, 9.95ml remaining..
INFO:root:Moving Pipette out of O2..
INFO:root:Moving to Mixing Chamber..
INFO:root:Dispense complete.
INFO:root:Moving to N2..
INFO:root:Pipette charged.
INFO:root:Dropping Pipette into N2..
INFO:root:Aspiration complete.
INFO:root:20.0ul extracted, 9.98ml remaining..
INFO:root:Moving Pipette out of N2..
INFO:root:Moving to Mixing Chamber..
INFO:root:Dispense complete.
INFO:root:Moving to CH4..
INFO:root:Pipette charged.
INFO:root:Dropping Pipette into CH4..
INFO:root:Aspiration complete.
INFO:root:200.0ul extracted, 9.8ml remaining..
INFO:root:Moving Pipette out of CH4..
INFO:root:Moving to Mixing Chamber..
INFO:root:Dispense complete.
INFO:root:Moving to NH4..
INFO:root:Pipette charged.
INFO:root:Dropping Pipette into NH4..
INFO:root:Aspiration complete.
INFO:root:100.0ul ext

,Name,Volume (uL),Starting Volume (mL),Aspirate Speed (mbar/s),Aspirate Constant (mbar/ml)
0,O2,50.0,9.950,1.0,1.0
1,N2,20.0,9.980,1.0,1.0
2,CH4,200.0,9.800,1.0,1.0
3,NH4,100.0,9.900,1.0,1.0
4,CO2,0.0,10.000,1.0,1.0
5,CH3CH3,70.0,9.930,1.0,1.0
6,OH,10.0,9.990,1.0,1.0
7,NaOH,0.0,10.000,1.0,1.0
8,PO4,25.0,9.975,1.0,1.0
9,Au,35.0,9.965,1.0,1.0
